# Pretraining on unlabeled data

This chapter covers training our model as well as using basic model evaluation techniques.

In [5]:
# Setting the previous LLM
import torch
from chapter04 import GPTModel

GPT_CONFIG_124M = {
 "vocab_size": 50257,
 "context_length": 256, # Reduced context length to just 256 tokens
 "emb_dim": 768,
 "n_heads": 12,
 "n_layers": 12,
 "drop_rate": 0.1,
 "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [3]:
# Functions for text to code
import tiktoken
from chapter04 import generate_text_simple

def text_to_token_ids(text, tokenizer):
 """Converts texts to token ids"""
 encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
 # Converts TokenIds into a Pytorch tensor
 encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
 return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
 "Changes ids to text"
 flat = token_ids.squeeze(0)
 return tokenizer.decode(flat.tolist()) # Converts back into text

# Sample text
start_context = "Every effort moves you"
# Initializes tokenizer
tokenizer = tiktoken.get_encoding("gpt2")
# Obtains the Token IDs
token_ids = generate_text_simple(
 model=model,
 idx=text_to_token_ids(start_context, tokenizer),
 max_new_tokens=10,
 context_size=GPT_CONFIG_124M["context_length"]
)
# Final generated text
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


As the generated text does not make any sense we have to train it, but first we will create a loss function to evaluate our model.

**Text generation loss**

In [ ]:
import torch
# Starting with a brief example
inputs = torch.tensor([[16833, 3626, 6100], # ["every effort moves",
 [40, 1107, 588]]) # "I really like"]

# Targets are inputs shifted one position forward
targets = torch.tensor([[3626, 6100, 345 ], # [" effort moves you",
 [1107, 588, 11311]]) # " really like chocolate"]

# Porviding the input to our model
with torch.no_grad(): # No training yet
 logits = model(inputs) # Output

probas = torch.softmax(logits, dim=-1) # Softmax
print(probas.shape)
# First number is the batch -> 2 (two example in the input)
# Second number is the number of tokens -> 3 (tokens in each input )
# Last number is the embedding dimensionality

# Obtaining best values
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
# Outputs one value per input token
print("Token IDs:\n", token_ids)

# Decoding the tokens
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1:"
 f" {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")


# Obtaining the respective probabilities
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)
text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

# We want to maximize the likelihood of the correct word appearing, which in other terms also means having a higher 
# probability with respect to other words

torch.Size([2, 3, 50257])
Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])
Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix
Text 1: tensor([7.4540e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([1.0337e-05, 5.6776e-05, 4.7559e-06])
